In [1]:
from sklearn.metrics import mean_squared_error, roc_auc_score
import numpy as np
import pandas as pd
import math

In [2]:
Train_ratings=pd.read_csv("/Users/dwj/CSCI553/HW/Project/yelp-recommendation-system/baseline/data/train_data_full.csv",encoding="ISO-8859-1")
Test_ratings = pd.read_csv("/Users/dwj/CSCI553/HW/Project/yelp-recommendation-system/baseline/data/ItemBasedCF.csv",encoding="ISO-8859-1")

In [6]:
User_mean =  Train_ratings.groupby(['userID'], as_index = False, sort = False).mean().rename(columns = {'rate': 'rating_mean'})[['userID','rating_mean']]

In [4]:
print(User_mean.head(10))
print(Test_ratings.size)
print(User_mean.size)

                   userID  rating_mean
0  msQe1u7Z_XuqjGoqhB0J5g          3.0
1  TlvV-xJhmh7LCwJYXkV-cg          5.0
2  NhOc64RsrTT1Dls50yYW8g          3.0
3  LKcdcMrq2xDzIThSHiZmLg          4.0
4  hxqo4NyJFfeOmuoVi--s1A          5.0
5  eG6HneK9zLcuZpVuKcsCGQ          5.0
6  5ngpW5tf3ep680eG1HxHzA          3.5
7  eAavwM32i2h7sNNer--kGw          1.0
8  nqzeKqdWBruvdgMd6aBHrg          5.0
9  Fc_nb6N6Sdurqb-rwsY1Bw          1.0
1799100
2692958


In [5]:
Data_merge = pd.merge(Test_ratings,User_mean,on = 'userID', how = 'inner', sort = False)

In [6]:
print(Data_merge.head(10))

                   userID              businessID      rate  rating_mean
0  B1xWMLMeUcu0jVT_R7W36g  B01pO_pI_8AOfEsukGNg-Q  3.053693     3.750000
1  J5n8LBC06H0Cg8jiYumIaA  X1S8u_cFfBpdWfvQBo_BqA  2.545512     2.666667
2  0tvCcnfJnSs55iB6mqPk3w  0d0i0FaJq1GIeW1rS2D-5w  3.587488     3.519164
3  0tvCcnfJnSs55iB6mqPk3w  RqAP37Oi5Oyo_ZvDg5vcQQ  3.480914     3.519164
4  0tvCcnfJnSs55iB6mqPk3w  7RTwUpuPFnIHPoTmWJpWhQ  3.463761     3.519164
5  0tvCcnfJnSs55iB6mqPk3w  GWNmYFQw3WitfTPt5uGMvQ  3.675031     3.519164
6  0tvCcnfJnSs55iB6mqPk3w  ND-zzXnjjZ_QjwtCttOOWg  4.013786     3.519164
7  0tvCcnfJnSs55iB6mqPk3w  qoiUo-7k2ATm0eokxNr8nA  3.143428     3.519164
8  0tvCcnfJnSs55iB6mqPk3w  m6QREZLAQatX1swZ2FN4hQ  3.461537     3.519164
9  0tvCcnfJnSs55iB6mqPk3w  95CCHowiE0W_ovv5-9lQAQ  3.544858     3.519164


In [7]:
Data_merge['y_true'] = Data_merge['rate'] - Data_merge['rating_mean']
print(Data_merge.head(10))

                   userID              businessID      rate  rating_mean  \
0  B1xWMLMeUcu0jVT_R7W36g  B01pO_pI_8AOfEsukGNg-Q  3.053693     3.750000   
1  J5n8LBC06H0Cg8jiYumIaA  X1S8u_cFfBpdWfvQBo_BqA  2.545512     2.666667   
2  0tvCcnfJnSs55iB6mqPk3w  0d0i0FaJq1GIeW1rS2D-5w  3.587488     3.519164   
3  0tvCcnfJnSs55iB6mqPk3w  RqAP37Oi5Oyo_ZvDg5vcQQ  3.480914     3.519164   
4  0tvCcnfJnSs55iB6mqPk3w  7RTwUpuPFnIHPoTmWJpWhQ  3.463761     3.519164   
5  0tvCcnfJnSs55iB6mqPk3w  GWNmYFQw3WitfTPt5uGMvQ  3.675031     3.519164   
6  0tvCcnfJnSs55iB6mqPk3w  ND-zzXnjjZ_QjwtCttOOWg  4.013786     3.519164   
7  0tvCcnfJnSs55iB6mqPk3w  qoiUo-7k2ATm0eokxNr8nA  3.143428     3.519164   
8  0tvCcnfJnSs55iB6mqPk3w  m6QREZLAQatX1swZ2FN4hQ  3.461537     3.519164   
9  0tvCcnfJnSs55iB6mqPk3w  95CCHowiE0W_ovv5-9lQAQ  3.544858     3.519164   

     y_true  
0 -0.696307  
1 -0.121154  
2  0.068324  
3 -0.038250  
4 -0.055402  
5  0.155868  
6  0.494622  
7 -0.375735  
8 -0.057627  
9  0.025695  


In [8]:
y_true = np.array(Data_merge['y_true'])

In [9]:
y_scores = np.array(Data_merge['rate'])

In [10]:
with np.nditer(y_true, op_flags=['readwrite']) as it:
    for x in it:
        if x >=0:
            x[...] = int(1)
        else:
            x[...] = int(0)

In [11]:
roc_auc_score(y_true, y_scores)

0.4612342569462242

In [18]:
Test_ratings_1 = pd.read_csv("/Users/dwj/CSCI553/HW/Project/yelp-recommendation-system/baseline/data/modelBasedCF_2.csv",encoding="ISO-8859-1")

In [19]:
Data_merge_1 = pd.merge(Test_ratings_1,User_mean,on = 'userID', how = 'inner', sort = False)

In [20]:
Data_merge_1['y_true'] = Data_merge_1['rate'] - Data_merge_1['rating_mean']

In [21]:
y_true_1 = np.array(Data_merge_1['y_true'])
y_scores_1 = np.array(Data_merge_1['rate'])

In [22]:
with np.nditer(y_true_1, op_flags=['readwrite']) as it:
    for x in it:
        if x >=0:
            x[...] = int(1)
        else:
            x[...] = int(0)

In [23]:
roc_auc_score(y_true_1, y_scores_1)

0.7331966576150378